In [ ]:
''' 
PAGE 1 

Le user arrive sur la page streamlit.

On propose une page de classement des films par :
* Ordre alphabétique
* par réalisateur
* par acteurs / actrices
* par genres
* par popularité (moyenne bayesian du rating)
* par année (et ordre alphabétique)
* par durée
* etc.

Tout ça constitue une première page du site.

------------------------------------------------------------------------------------------------------------------------------------------------------------------
Item-Item = kNN / User-Item : SVD / Deep

PAGE 2

Sur une seconde page, on propose une catégorie : mes films / ma bilbiothèque, etc. whatever --> Recommandations personnelles.

CAS 1 - L'utilisateur novice : Donne la liste de ses genres de films préférés. Eventuellement acteurs / actrices / (réalisateurs?). 
On crée un film fictif --> similarité cosinus) --> On sort les k films les plus proches. On les notes + mieux notés bayesian.

FAIT


CAS 2 -  L'utilisateur medium : Il donne un film (ou plusieurs) qu'il aime. En fonction de ce qu'il donne on complète la liste avec du collaborative 
filtering item-item kNN. et l'algo lui renvoie une liste des k films qu'il aimera.
 

 
CAS 3 -  L'utilisateur cinéphile : Fait une notation sur un échantillon conséquent de films (de son choix ? qu'on lui propose). A partir de ses notations, on sort les meilleurs films. 
Approche deep learning simple : collaborative filtering user-item ou
va aimer. --> collaborative filtering (Deep learning ou SVD)



(Possibilité de faire de l'hybride dans toutes les cas, mais il faut à chaque fois pondérer différemment)

Les k plus proches voisins sont une méthode de base pour le collaborative filtering basée sur du item-item. A la place, on peut utiliser la factorisation
matricielle avec SVD, ou encore le deep learning.

Pour le content based filtering, on se contente de la matrice de similarité cosinus.




On récupère toutes ces informations.

On cherche les k plus proches voisins des films donnés 
On cherche les k films les mieux notés qui pro



 '''

### PREPROCESSING 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
links = pd.read_csv('links.csv')

title_principals = pd.read_csv('data-7.tsv', sep = '\t') # tconst	ordering	nconst	category	job	characters 38.3s

In [ ]:
# faire le lien avec la base imdb

movies = movies.merge(right = links, on = 'movieId', how = 'left')
movies.imdbId = movies.imdbId.apply(lambda x : 'tt' + (7 - len(str(x))) * str(0) + str(x))
movies = movies.drop(columns='tmdbId')
movies = movies.rename(columns = {'imdbId' : 'tconst'})
# Le lien est fait avec la base imdb

In [ ]:
# Hypothèses
nombre_de_films = 1500
nb_users = 1500

In [ ]:
# On crée un DataFrame movie_stats qui contient pour chaque ligne le film et la moyenne bayesian des notations

movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

C = movie_stats['count'].mean()
m = movie_stats['mean'].mean()

def bayesian_avg(ratings):
    bayesian_avg = (C*m+ratings.sum())/(C+ratings.count())
    return bayesian_avg

bayesian_avg_ratings = ratings.groupby(['movieId'])['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
movie_stats = movie_stats.merge(bayesian_avg_ratings, on='movieId')

movie_stats = movie_stats.merge(movies['movieId'])
best_movies_sorted = movie_stats.sort_values('bayesian_avg', ascending=False) 
# On obtient le classement des meilleurs films avec la moyenne bayesian, c'est à dire la moyenne des notations 
# pondérée du nombre de votes

In [ ]:
# on réduit le DataFrame movies en ne gardant que les k (= nombre_de_films) films les mieux notés

movies_reduced = movies.merge(right = best_movies_sorted[:nombre_de_films], on = 'movieId', how = 'right')

# on enlève la moyenne normale et le décompte

movies_reduced = movies_reduced.drop(columns=['mean', 'count'])

# Notre base de donnée de films est prête pour utilisation

In [ ]:
# on diminue le dataframe ratings avec prise en compte que des k folms de movies_reduced - 2.5s

ratings_reduced = movies_reduced.merge(ratings) 

# on diminue ratings_reduced en ne gardant que les p users ayant effectué le plus de notations
liste = list(ratings_reduced.groupby('userId').agg('count').sort_values(by = 'rating', ascending = False)[:nb_users].index) 
ratings_reduced = ratings_reduced[ratings_reduced['userId'].isin(liste)]

In [ ]:
# On réindexe les matrices movies_reduced et rating_reduced

# Création d'un dico pour faire des nouveaux id aux items + création nouvelle colonne au DataFrame
liste_movieid = list(movies_reduced.movieId.unique())
new_movieid = [i for i in range(len(movies_reduced.movieId.unique()))]
dico_movie = dict(zip(liste_movieid, new_movieid))

# Création d'un dico pour faire des nouveaux id aux user_id + création nouvelle colonne au DataFrame
liste_userid = list(ratings_reduced.userId.unique())
new_userid = [i for i in range(len(ratings_reduced.userId.unique()))]
dico_user = dict(zip(liste_userid, new_userid))


# Il est primordial de comprendre ici qu'il ne faut pas faire de boucle !!! Il faut utiliser les super propriétés de pandas. 
# Ici la boucle tournerait en énormément de temps, alors que là c'est quasiment instantané.

movies_reduced['new_movieid'] = movies_reduced.movieId.apply(lambda x : dico_movie[x])
ratings_reduced['new_movieid'] = ratings_reduced.movieId.apply(lambda x : dico_movie[x])
ratings_reduced['new_userid'] = ratings_reduced.userId.apply(lambda x : dico_user[x])



# On change la position des colonnes du df pour se remettre dans la configuration voulue
movies_reduced = movies_reduced.reindex(columns=["new_movieid",  "title", "genres", "tconst", "bayesian_avg"])
movies_reduced = movies_reduced.rename(columns={'new_movieid' : 'movieId'})
ratings_reduced = ratings_reduced.reindex(columns=["new_userid", "new_movieid", "rating"])
ratings_reduced = ratings_reduced.rename(columns={'new_movieid' : 'movieId', 'new_userid' : 'userId'})
ratings_reduced = ratings_reduced.reset_index()
ratings_reduced = ratings_reduced.drop(columns='index')




### Content-Based Filtering

In [ ]:
# cbf -> content_based filtering
movies_cbf = movies_reduced.drop(columns=['bayesian_avg'])

In [ ]:
#on reste plutôt cablée film américain. On assume ce choix, quitte à un jour faire une version française. Mais on a classé les films par 
#la moyenne bayesian ce qui nous laisse forcément que des films américains. Donc on regarde les actors dans le top monde.

from bs4 import BeautifulSoup as bs
import requests

url_actors = 'https://www.imdb.com/list/ls050274118/'
url_actresses = 'https://www.imdb.com/list/ls000055475/'
url_directors = 'https://www.imdb.com/list/ls053823383/'
page_actors = requests.get(url_actors)
page_actresses = requests.get(url_actresses)
page_directors = requests.get(url_directors)

soup_actors = bs(page_actors.content, "lxml")
soup_actresses = bs(page_actresses.content, "lxml")
soup_directors = bs(page_directors.content, "lxml")

actor_ids = soup_actors.find_all('div', class_ ='lister-item-image')
actress_ids = soup_actresses.find_all('div', class_ ='lister-item-image')
director_ids = soup_directors.find_all('div', class_ ='lister-item-image')

list_best_actors = []
list_best_actresses = []
list_best_directors = []


for i in range(len(actor_ids)):
    list_best_actors.append(actor_ids[i].find('a')['href'].split('/')[2])

for i in range(len(actress_ids)):
    list_best_actresses.append(actress_ids[i].find('a')['href'].split('/')[2])

for i in range(len(director_ids)):
    list_best_directors.append(director_ids[i].find('a')['href'].split('/')[2])

# on repère les doublons éventuels

for i in list_best_actors:
    for j in list_best_directors:
        if i == j:
            list_best_directors.remove(j)
            break

for i in list_best_actresses:
    for j in list_best_directors:
        if i == j:
            list_best_directors.remove(j)
            break
        
#TIME -  12s

In [ ]:
# traitement du dataset movies

movies_cbf.genres = movies_cbf.genres.apply(lambda x : x.split('|'))


In [ ]:
# on lie movies_reduced avec les principaux personnages de chaque film.
movies_cbf = movies_cbf.merge(right = title_principals, on = 'tconst', how = 'left') #15s
movies_cbf = movies_cbf.drop(columns=['job', 'characters'])

In [ ]:
liste_iter = []
liste_globale = []

for i in range(nombre_de_films) :
    for j in movies_cbf[movies_cbf.movieId == i].nconst :
        liste_iter.append(j)
    liste_globale.append(liste_iter)
    liste_iter = []

In [ ]:
movies_cbf['people'] = movies_cbf.movieId.apply(lambda x : liste_globale[x])
movies_cbf

In [ ]:
movies_cbf = movies_cbf.drop(columns=['nconst', 'ordering', 'category'])
movies_cbf

In [ ]:
movies_cbf = movies_cbf.drop_duplicates(subset=['movieId']).reset_index().drop(columns='index')
movies_cbf

In [ ]:
# Des personnages principaux on ne garde que les actor / actress / director

#movies_cbf = movies_cbf[(movies_cbf.category == 'director') | (movies_cbf.category == 'actor') | (movies_cbf.category == 'actress')]


In [ ]:
movies_cbf = movies_cbf.drop(columns=['movieId', 'title', 'tconst'])
movies_cbf

In [ ]:

#movies.genres = movies.genres.apply(lambda x : x.split('|')) - 35s
lis_genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime','Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', '(no genres listed)']

lis_totale = list_best_actors + list_best_actresses + list_best_directors


for i in lis_genres:
    movies_cbf[i] = 0


In [ ]:

for i in movies_cbf.columns[9:]:
    for j in movies_cbf.index:
        for k in movies_cbf.genres[j]:
            movies_cbf.loc[j,k] = 1 if k in lis_genres else 0


In [ ]:
movies_cbf = movies_cbf.drop(columns='genres')
movies_cbf

In [ ]:
#movies.genres = movies.genres.apply(lambda x : x.split('|')) - 35s

lis_totale = list_best_actors + list_best_actresses + list_best_directors


for i in lis_totale:
    movies_cbf[i] = 0


In [ ]:
# 25s
for i in movies_cbf.columns:
    for j in movies_cbf.index:
        for k in movies_cbf.people[j]:
            if k in lis_totale :
                movies_cbf.loc[j,k] = 1  
            else :
                pass


In [ ]:
movies_cbf

In [ ]:
movies_cbf = movies_cbf.drop(columns = 'people')
movies_cbf

In [ ]:
movies_reduced[movies_reduced['title'].str.contains('Khaled')] # Code à conserver


In [ ]:
# movies_reduced[movies_reduced['title'].str.contains('Godfather')] # Code à conserver

# Le code ci-dessous permet de tester le modèle avec différents films

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(movies_cbf, movies_cbf)
print(f"Dimensions of our genres cosine similarity matrix: {cosine_sim.shape}")

# Code permettant de trouver le nom d'un film en fonction de son 

def movie_finder(title):
    return movies_reduced[movies_reduced['title'].str.contains(title)]['title'].tolist()

movie_idx = dict(zip(movies_reduced['title'], list(movies_reduced.index))) # dictionnaire clé = titre, valeur = index du titre
title = movie_finder('Django')[0]
n_recommendations = 10

idx = movie_idx[title] # index du titre du film
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:(n_recommendations+1)]
similar_movies = [i[0] for i in sim_scores]

print(f"Recommendations for {title}:")
movies_reduced['title'].iloc[similar_movies]

# ATTENTION A LA RÉINDEXATION. IL FAUT S'ASSURER QUE CELA NE PERTURBE PAS LA SUITE DES OPÉRATIONS


### Cas n°1

In [ ]:
Action = 1
Adventure = 1
Animation = 0
Children = 0
Comedy = 0
Crime = 0
Documentary = 0
Drama = 0
Fantasy = 0
Film_Noir = 0
Horror = 0
IMAX = 1
Musical = 0
Mystery = 0
Romance = 0
Sci_Fi = 0
Thriller = 1
War = 0
Western = 0
no_genres_listed = 0


new_line = pd.DataFrame([[Action, Adventure, Animation, Children, Comedy, Crime,Documentary, Drama, Fantasy, Film_Noir, Horror, IMAX, 
              Musical, Mystery, Romance, Sci_Fi, Thriller, War, Western, no_genres_listed]], columns=['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime','Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', '(no genres listed)'])

# Fusion des deux DataFrame
movies_cbf_user = pd.concat([movies_cbf.iloc[:,:20], new_line], ignore_index=True)
movies_cbf_user

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity # La matrice cosinus permet d'évaluer le degré de similarité entre 2 vecteurs.

cosine_sim = cosine_similarity(movies_cbf_user, movies_cbf_user)
print(f"Dimensions of our genres cosine similarity matrix: {cosine_sim.shape}")

n_recommendations = 10

title = 'Visiteur'
idx = nombre_de_films # index du titre du film. A REMPLACER PAR UNE VARIABLE

sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = list(filter(lambda x: x[0] != nombre_de_films, sim_scores)) # on enlève de la liste le tuple correspondant au film de base
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
similar_movies = [i[0] for i in sim_scores]


print(f"Recommendations for {title}:")
movies_reduced['title'].iloc[similar_movies]

# ATTENTION A LA RÉINDEXATION. IL FAUT S'ASSURER QUE CELA NE PERTURBE PAS LA SUITE DES OPÉRATIONS
# Les films qui sortent sont classés, par construction, suivant leur moyenne bayesian

### Cas n°2

In [ ]:
from scipy.sparse import csr_matrix # L'idée ici est de générer une matrice de notations avec les users en ligne et les films en colonne

def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.

    Args:
        df: pandas dataframe containing 3 columns (userId, movieId, rating)

    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    M = df['userId'].nunique()  # nombre de users
    N = df['movieId'].nunique() # nombre de films - On définit ainsi la dimension de la matrice

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (user_index,item_index)), shape=(M,N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings_reduced)

In [ ]:
# Calcul of Matrix sparcity. La sparse matrix est une matrice où la plupart des éléments sont des zéros

n_total = X.shape[0]*X.shape[1] #Nombre total d'inputs de la matrice
n_ratings = X.nnz               # on repère les inputs non nuls de la matrice
sparsity = n_ratings/n_total    # On calcul le taux de remplissage de la matrice
print(f"Matrix sparsity: {round(sparsity*100,2)}%") 

In [ ]:
n_ratings_per_user = X.getnnz(axis=1) # On récupère le nombre de votes en ligne, c'est à dire par user

print(f"Most active user rated {n_ratings_per_user.max()} movies.")
print(f"Least active user rated {n_ratings_per_user.min()} movies.")

In [ ]:
n_ratings_per_movie = X.getnnz(axis=0) # On récupère le nombre de votes en colonne, c'est à dire par film

print(f"Most rated movie has {n_ratings_per_movie.max()} ratings.")
print(f"Least rated movie has {n_ratings_per_movie.min()} ratings.")

In [ ]:
# normalisation des données. Les données nulles sont remplacées par l'opposé des moyennes. POURQUOI ?

sum_ratings_per_movie = X.sum(axis=0)
mean_rating_per_movie = sum_ratings_per_movie/n_ratings_per_movie
X_mean_movie = np.tile(mean_rating_per_movie, (X.shape[0],1))
X_norm = X - csr_matrix(X_mean_movie) # 14s

print("Original X:", X[0].todense())
print("Normalized X:", X_norm[0].todense())

In [ ]:
# CONSTRUCTION DE L'ALGORITHME KNN
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    """
    Finds k-nearest neighbours for a given movie id.

    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations

    Output: returns list of k similar movie ID's
    """
    X = X.T
    neighbour_ids = []

    movie_ind = movie_mapper[movie_id]  # on cherche l'indice de ce film dans la matrice
    movie_vec = X[movie_ind]            # on trouve ainsi le vecteur de notations correspondant à ce film
    if isinstance(movie_vec, (np.ndarray)): # on cherche à savoir si movie vec est un array ???
        movie_vec = movie_vec.reshape(1,-1)
    # use k+1 since kNN output includes the movieId of interest
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [ ]:

# SORTIES DE L'ALGORITHME KNN -- 1 min 30
# sur quelle base est-ce que knn donne des résultats : Il va chercher dans la matrice X, où on a remplacé les nulle par des - moyenne. 
#Il va chercher les ratings les plus proches, tout simplement. On peut dire que son approche prend en compte le nombre de votes puisque 
# tous les votes nuls sont pénalisants.
# Néanmmoins, l'approche kNN sur du filtrage collaboratif simple revient à classer avec une moyenne bayesian.
# Il n'est pas possible d'évaluer cette approche,  puisqu'on ne cherche pas ici à prédire une note, on cherche simplement les k voisins. 


movie_titles = dict(zip(movies_reduced['movieId'], movies_reduced['title'])) # On associe les movieId avec leur titre dans un dictionnaire


In [ ]:
movies_reduced[movies_reduced['title'].str.contains('Godfather')] # Code à conserver

In [ ]:

movie_id = 1 

similar_movies = find_similar_movies(movie_id, X_norm, movie_mapper, movie_inv_mapper, metric='cosine', k=10)

movie_title = movie_titles[movie_id]

print(f"Because you watched {movie_title}:")
for i in similar_movies:
    print(movie_titles[i]) # problème d'index

# Faire une critique du résultat obtenu par kNN en le comparant à un classement bayesian, et ensuite passer au content-based Filtering qui
# devrait nous permettre de faire des évaluations.

# Tourne en  

# AJOUTER LES DISTANCES 


### Cas n°3

In [ ]:
# On se base sur un profil d'utilisateur qui va entrer des notes et rentrer dans la base de notation
# On propose l'utilisateur 

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dot, Dense
from sklearn.model_selection import train_test_split


In [ ]:
# Train, Test, split
train, test = train_test_split(ratings_reduced, test_size=0.2, random_state=42)

# Nombre d'éléments uniques
num_users = ratings_reduced['userId'].nunique()
num_movies = ratings_reduced['movieId'].nunique()

# Modèle
user_input = Input(shape=(1,))
user_embedding = Embedding(input_dim=num_users, output_dim=10)(user_input)
user_flatten = Flatten()(user_embedding)

item_input = Input(shape=(1,))
item_embedding = Embedding(input_dim= num_movies, output_dim=10)(item_input)
item_flatten = Flatten()(item_embedding)

dot_product = Dot(axes=1)([user_flatten, item_flatten])

# Add a dense layer for final prediction
hidden_layer = Dense(64, activation='relu')(dot_product)
output_layer = Dense(1)(hidden_layer)

model = Model(inputs=[user_input, item_input], outputs=output_layer)

model.compile(loss='mean_squared_error', optimizer='adam', metrics = ["mae"])

model.fit([train['userId'], train['movieId']], train['rating'], epochs=3, batch_size=32, validation_data=([test['userId'], test['movieId']], test['rating']))


In [ ]:

# Make predictions
predictions = model.predict([test['userId'], test['movieId']])
pd.DataFrame(predictions).describe()


In [ ]:
import numpy as np 
import pandas as pd 
from surprise import Dataset 
from surprise import Reader
from surprise.model_selection import train_test_split 
from surprise.model_selection import cross_validate 
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise import NormalPredictor 
from surprise import BaselineOnly 
from surprise import KNNBasic
from surprise import KNNWithMeans 
from surprise import KNNWithZScore 
from surprise import KNNBaseline
from surprise import SVD 
from surprise import SVDpp 
from surprise import NMF

In [ ]:
ratings_reduced

In [ ]:
#column_names = ['user_id', 'item_id', 'rating', 'timestamp']
#df = pd.read_csv('u.data', sep='\t', names=column_names)

#movie_titles = pd.read_csv("Movie_Id_Titles")

#df = pd.merge(df,movie_titles,on='item_id')

n_users = ratings_reduced.userId.nunique()
n_movies = ratings_reduced.movieId.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_movies))

#df = df.rename(columns={'user_id' : 'userId', 'item_id' : 'movieId'})


# Call Reader and set Rating Scale from 0.5 to 5
reader = Reader(rating_scale= (0.5, 5))
# Parse data and select only necessary files
data = Dataset.load_from_df(ratings_reduced[['userId', 'movieId','rating']], reader)


In [ ]:

# Split data into 75% / 25%
trainset, testset = train_test_split(data, test_size=.2)


In [ ]:

# Run svd algroichm
model = SVD()
# This follows the typical sklearn train and test model building
model.fit(trainset)
predictions = model.test(testset)
# compute erros accuracy.rmse(predictions)
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)
accuracy.fcp(predictions)

# 34 min sur data frame complet, RMSE 0.7889